# Read in data

In [ ]:
import pandas as pd

dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset

In [ ]:
# Convert columns to numeric, ignoring errors to skip any non-convertible values
cols_to_convert = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                   'Global_intensity', 'Sub_metering_1', 'Sub_metering_2']

for col in cols_to_convert:
    dataset[col] = pd.to_numeric(dataset[col], errors='coerce')

In [ ]:
dataset.info()

In [ ]:
dataset

# Aggregate data into daily

In [ ]:
df = dataset.resample('D').sum()

In [ ]:
df

# Evaluation Metric for the predictions

Generate an overall RMSE score (i.e., average for the week) and an individual RMSE score fore each forecast (i.e., for each day in the week)

In [ ]:
import numpy as np  # This is generally used for handling arrays
from sklearn.metrics import mean_squared_error  # For calculating MSE
from math import sqrt  # For calculating the square root

def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# Build train and test sets

Train --> first 3 years \
Test --> last 1 year


To frame the problem, the data is broken down into standard weeks i.e. begin on a sunday and end on a saturday.

In [ ]:
from numpy import array, split

# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[1:-328], data[-328:-6]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test

In [ ]:
train, test = split_dataset(df.values)
# validate train data
print(train.shape)
print(train[0, 0, 0], train[-1, -1, 0])
# validate test
print(test.shape)
print(test[0, 0, 0], test[-1, -1, 0])

The results above make sense as the training is 3 years (i.e., 159 weeks) and the testing is 1 year (i.e., 46 weeks)

(n, q ,r) \
n = the week  \
q = the specific day \
r = the values of the columns for that day i.e., power consumption etc.

# Validation - using 'Walk-Forward'

k-fold cross valdidation does not work for time series data, instead 'walk-forward' is used.

This is where a model is required to make a one week prediction, then the actual data for that week is made available to the model so that it can be used as the basis for making a prediction on the subsequent week. This is both realistic for how the model may be used in practice and beneficial to the models allowing them to make use of the best available data.

In [ ]:
# evaluate a single model
def evaluate_model(train, test, n_input, n_output):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# Build univariate LSTM

The LSTM model expects data to be in the following format: \
[samples, timesteps, features] 

samples = number of weeks \
timesteps = day of the week \
features = power consumption 

For the training data it would be: \
[159, 7, 1]


### Reformat the data for use input into the LSTM

A problem is that 159 instances is not a lot to train a neural network. A way to create a lot more training data is to change the problem during training to predict the next seven days given the prior seven days, regardless of the standard week. For example:

Input, Output \
[d01, d02, d03, d04, d05, d06, d07], [d08, d09, d10, d11, d12, d13, d14] \
[d02, d03, d04, d05, d06, d07, d08], [d09, d10, d11, d12, d13, d14, d15]

In [ ]:
# flatten data
data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
data.shape

Below is function that takes the list of weeks and turns the time series data into a supervised learning format in a moving window.

In [ ]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

In [ ]:
# n_input dictates how many prior days we want to consider in predicting the next n_output days
n_input = 14
n_output = 7

X, y = to_supervised(train, n_input, n_output)

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
y.shape

In [ ]:
y[0]

# Build the LSTM model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 

# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 70, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# Make predictions using the fitted LSTM model

The model expects data to have the same three dimensional shape when make a prediction i.e. \

[1, 7, 1] 

1 = 1 sample \
7 = 7 days (depends on your n_input) \
1 = 1 feature

In order to predict the next week, we need to retrieve the last days of observations. As with the training data we need to flatten the history data to remove the weekly structure.

In [ ]:
# retrieve last observations for input data
input_x = data[-n_input:, 0]
input_x

In [ ]:
# reshape into [1, n_input, 1]
input_x_reshaped = input_x.reshape((1, len(input_x), 1))
input_x_reshaped.shape

Make the forecast

In [ ]:
# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, 0]
	# reshape into [1, n_input, 1]
	input_x = input_x.reshape((1, len(input_x), 1))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat